In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Data reading

In [0]:
df = spark.read.format("parquet")\
                .load("abfss://bronze@etedatabricksa.dfs.core.windows.net/Products")

In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

### Functions

In [0]:
df.createOrReplaceTempView("Products")

In [0]:
%sql
DROP VIEW IF EXISTS databricks_catalog.bronze.products;


In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.discount_func(p_price double)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90;

In [0]:
%sql
select product_id,price,databricks_catalog.bronze.discount_func(price) as discounted_price
from Products;

In [0]:
df = df.withColumn("discounted_price", expr("databricks_catalog.bronze.discount_func(price)"))

df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION DATABRICKS_CATALOG.BRONZE.UPPER_FUNC(P_BRAND_STRING STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return P_BRAND_STRING.upper()
$$

In [0]:
%sql
select product_id,brand, DATABRICKS_CATALOG.BRONZE.UPPER_FUNC(brand) as brand_upper
from Products;

In [0]:
df.write.format("delta")\
            .mode("append")\
            .option("path","abfss://silver@etedatabricksa.dfs.core.windows.net/products")\
            .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.product_silver
USING DELTA
LOCATION 'abfss://silver@etedatabricksa.dfs.core.windows.net/products'